In [21]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [22]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE'%github.com%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
print(missing_data)

      repo_id                         repo_name  \
0       28026            network-config-manager   
1       30418  streaming-http-adapter-buildpack   
2       30446                             nsx-t   
3       30440        cloud-suitability-analyzer   
4       30434        rabbitmq-oauth-example-app   
...       ...                               ...   
1771    28063                      git-resource   
1772    28143                   harbor-operator   
1773    28024            terraform-provider-vmc   
1774    26440                          docs-ksm   
1775    27938           terraform-provider-vra7   

                                               repo_git  max_pr_count  \
0      https://github.com/vmware/network-config-manager            58   
1     https://github.com/projectriff/streaming-http-...            21   
2               https://github.com/vmware-samples/nsx-t             8   
3     https://github.com/vmware-samples/cloud-suitab...            12   
4     https://github.c

In [23]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'last_pr_collected', 'pull_requests_missing', 'ratio_abs', 'repo_git']]
print(df1)
missing_data.to_csv('/Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/output/missing.csv')

     repo_id  repo_id   last_pr_collected  pull_requests_missing  ratio_abs  \
0      28026    28026 2020-10-04 16:52:51                     33   0.440678   
1      30418    30418 2020-10-29 18:00:19                      8   0.636364   
2      30446    30446 2020-10-19 23:58:08                      3   0.666667   
3      30440    30440 2020-10-04 18:58:34                      4   0.692308   
4      30434    30434 2020-10-20 01:34:28                      3   0.727273   
..       ...      ...                 ...                    ...        ...   
267    27866    27866 2020-09-29 15:59:57                      1   0.997630   
268    30117    30117 2020-07-29 00:33:22                      2   0.997817   
269    25695    25695 2020-10-20 06:23:04                      1   0.998246   
270    27907    27907 2020-09-29 16:09:49                      2   0.998653   
271    27892    27892 2020-06-02 01:08:33                      1   0.999749   

                                              repo_